<a href="https://colab.research.google.com/github/AliEdalat/ChatBot_for_persian_LLaMA_fine_tune/blob/main/ChatBot_for_persian_LLaMA_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # Install FastChat
# !pip3 install fschat

# # Install the latest main branch of huggingface/transformers
# !pip3 install git+https://github.com/huggingface/transformers

In [2]:
!git clone https://github.com/lm-sys/FastChat.git
%cd FastChat

Cloning into 'FastChat'...
remote: Enumerating objects: 1142, done.
remote: Counting objects: 100% (1142/1142), done.
remote: Compressing objects: 100% (452/452), done.
remote: Total 1142 (delta 722), reused 1034 (delta 652), pack-reused 0
Receiving objects: 100% (1142/1142), 28.21 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (722/722), done.
/content/FastChat


In [3]:
# !pip install transformers

In [4]:
!pip3 install --upgrade pip  # enable PEP 660 support
!pip3 install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-_5u_w7l3/transformers_cac7e3765f7c44cabaa81e3a9149141e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-_5u_w7l3/transformers_cac7e3765f7c44cabaa

In [5]:
!pip install einops
!pip install peft
# !pip install flash-attn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00


In [6]:
# bf16 and tf32 for new gpu, per_node = num of gpu

In [7]:
# !torchrun --nnodes=1 --nproc_per_node=1 --master_port=4545 \
#     fastchat/train/train_mem.py \
#     --model_name_or_path 'decapoda-research/llama-7b-hf' \
#     --data_path './playground/data/alpaca-data-conversation.json' \
#     --bf16 False \
#     --output_dir ./checkpoints \
#     --num_train_epochs 3 \
#     --per_device_train_batch_size 1 \
#     --per_device_eval_batch_size 4 \
#     --gradient_accumulation_steps 1 \
#     --evaluation_strategy "no" \
#     --save_strategy "steps" \
#     --save_steps 1200 \
#     --save_total_limit 100 \
#     --learning_rate 2e-5 \
#     --weight_decay 0. \
#     --warmup_ratio 0.03 \
#     --lr_scheduler_type "cosine" \
#     --logging_steps 1 \
#     --fsdp "full_shard auto_wrap" \
#     --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
#     --tf32 False \
#     --model_max_length 2048 \
#     --gradient_checkpointing True \
#     --lazy_preprocess True

In [8]:
# !cat /usr/local/lib/python3.9/dist-packages/transformers/training_args.py

In [9]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_size = "large"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-translation_en_fa"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)



def run_MT_model(input_string, **generator_args):
    generator_args = {
    "max_length": 512,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    print(output)
    return output

In [10]:
# _ = run_MT_model("i am disabled in iran. i want to follow iran instructions. it is very nice but it is lazy")
# run_MT_model("shrouds herself in white and walks penitentially disguised as brotherly love through factories and parliaments; offers help, but desires power;")
# run_MT_model("He thanked all fellow bloggers and organizations that showed support.")
# run_MT_model("Races are held between April and December at the Veliefendi Hippodrome near Bakerky, 15 km (9 miles) west of Istanbul.")
run_MT_model("I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?")

['من می خواهم پایان نامه دکتری را در رشته علوم کامپیوتر در مورد شبکه اجتماعی دنبال کنم، مشکل باز در شبکه های اجتماعی چیست؟']


['من می خواهم پایان نامه دکتری را در رشته علوم کامپیوتر در مورد شبکه اجتماعی دنبال کنم، مشکل باز در شبکه های اجتماعی چیست؟']

In [11]:
import pandas as pd
from pandas.io.json import json_normalize
import json

# pd.json_normalize(json.load(open('./playground/data/alpaca-data-conversation.json')), record_path=[['conversations']])
all = json.load(open('./playground/data/alpaca-data-conversation.json'))

for i in range(len(all)):
  for j in range(len(all[i]['conversations'])):
    temp = all[i]['conversations'][j]['value']
    print(temp)
    break
  break

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:


In [12]:
!git clone https://github.com/thisserand/alpaca-lora-finetune-language.git

Cloning into 'alpaca-lora-finetune-language'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 18), reused 37 (delta 13), pack-reused 0
Unpacking objects: 100% (46/46), 18.21 MiB | 4.87 MiB/s, done.


In [13]:
%cd /content/FastChat/alpaca-lora-finetune-language

/content/FastChat/alpaca-lora-finetune-language


In [14]:
import json
import pandas as pd
import numpy as np
import re
import glob

input_tasks_path = "/content/FastChat/alpaca-lora-finetune-language/data/source_tasks/tasks_not_translated_en.json"

with open(input_tasks_path, "rb") as f:
    json_data = json.loads(f.read())
    df = pd.DataFrame(json_data)
    
def write_json_file(blob, file_path):
    with open(file_path, 'w') as file:
            json.dump(blob, file)

In [15]:
def matches_regex(regex, text):
    return bool(re.compile(regex).search(text))


def contains_code(text):
    # filter based on keywords that indicate code
    code_blacklist = ['&&', '||', '<html>', ';\n', 'SELECT']
    
    return (
            any(code_keyword in text for code_keyword in code_blacklist) |
            matches_regex(r'\w+\(\w*\) \{', text) | # e.g. myFunc() {
            matches_regex(r'def \w+\(', text) | # e.g. def parse_list(
            matches_regex(r'\[A-z]+\.[A-z]+', text) | # e.g. this.language
            matches_regex(r': [\w\.#]{1,12};', text) | # e.g. font-size: 1.3em;
            matches_regex(r'<\/\w+>', text) # e.g. </html>
           )


def contains_words(text):
    return matches_regex(r'[A-z]{3,}', text) # words with at least three characters


def is_translatable(text):
    if text == "":
        return True # empty string won't be charged by DeepL
    return (contains_code(text) is False) & contains_words(text)

In [16]:
def translate_list(text_list):
    
    return [run_MT_model(response)[0] for response in text_list]


def translate_and_update_series(text_series):
    # memorize whether and where the list contains non-translatable content
    is_translatable_index = text_series.apply(lambda x: is_translatable(x) is False)
    text_list_source_language = text_series.tolist()

    # replace non-translatable content with an empty string
    text_series[is_translatable_index] = ""

    # translate list
    text_list = text_series.tolist()
    
    translated_list = translate_list(text_list)

    # if list contains non-translatable content, replace accordingly
    if is_translatable_index.sum() > 0:
        for index, text_is_translatable in enumerate(is_translatable_index.tolist()):
            if text_is_translatable:
                translated_list[index] = text_list_source_language[index]
    return translated_list

In [17]:
import os

# Feel free to increase the chunk size. I was worried that the execution would be interrupted,
# so I used a smaller chunk size
chunk_size = 1
output_dir = './data/output/'

def translate_dataframe(df):
    os.makedirs(output_dir, exist_ok=True)
    number_of_chunks = df.shape[0] // chunk_size
    chunked_df_list = np.array_split(df, number_of_chunks)
    
    start_index = 1
    
    for index, chunk_df in enumerate(chunked_df_list[start_index:]):
        instruction_list_translated = translate_and_update_series(chunk_df.instruction)
        input_list_translated = translate_and_update_series(chunk_df.input)
        output_list_translated = translate_and_update_series(chunk_df.output)
        
        translated_df = pd.DataFrame({'instruction': instruction_list_translated, 'input': input_list_translated, 'output': output_list_translated})
        translated_dict = translated_df.to_dict('records')
        
        write_json_file(translated_dict, f'{output_dir}chunk{start_index+index}.json')

In [ ]:
translate_dataframe(df)

['سه رنگ اصلی چیست؟']
['برای اینکه این کار را بکنم']
['سه رنگ اصلی عبارتند از قرمز، آبی و زرد.']
['ساخت یک اتم را شرح دهید.']
['برای اینکه این کار را بکنم']
['یک اتم از یک هسته تشکیل می شود که حاوی پروتون ها و نترون ها و در محاصره ی الکترونها قرار دارد که در مدارهای اطراف هسته حرکت می کنند. پروتونها و نتران ها جرم مثبت دارند، در حالی که الکترونها جرم منفی دارند. تعداد هر ذره رقم اتمی و نوع اتم را مشخص می کند.']
['چگونه می توانیم آلودگی هوا را کاهش دهیم؟']
['برای اینکه این کار را بکنم']


In [ ]:
def combine_chunks():
    translated_tasks_list = []
    for index in range(0, len(glob.glob(f'{output_dir}*.json'))):
        with open(f'{output_dir}chunk{index}.json', "rb") as f:
            translated_tasks_list += json.loads(f.read())
    write_json_file(translated_tasks_list, f'./translated_tasks_fa_{TRANSLATOR}.json')

combine_chunks()

In [ ]:
%cd /content/FastChat/alpaca-lora-finetune-language/data/output
!git init
!git config --global user.email "ali.edalat.77@gmail.com"
!git config --global user.name "AliEdalat"
!git add -A
!git commit -m "fa_data"
!git remote add origin https://github.com/AliEdalat/ChatBot_for_persian_LLaMA_fine_tune.git
!git push -u origin